```
--- Day 16: Reindeer Maze ---
It's time again for the Reindeer Olympics! This year, the big event is the Reindeer Maze, where the Reindeer compete for the lowest score.

You and The Historians arrive to search for the Chief right as the event is about to start. It wouldn't hurt to watch a little, right?

The Reindeer start on the Start Tile (marked S) facing East and need to reach the End Tile (marked E). They can move forward one tile at a time (increasing their score by 1 point), but never into a wall (#). They can also rotate clockwise or counterclockwise 90 degrees at a time (increasing their score by 1000 points).

To figure out the best place to sit, you start by grabbing a map (your puzzle input) from a nearby kiosk. For example:

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
There are many paths through this maze, but taking any of the best paths would incur a score of only 7036. This can be achieved by taking a total of 36 steps forward and turning 90 degrees a total of 7 times:


###############
#.......#....E#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#..>>>>>>>>v#^#
###^#.#####v#^#
#>>^#.....#v#^#
#^#.#.###.#v#^#
#^....#...#v#^#
#^###.#.#.#v#^#
#S..#.....#>>^#
###############
Here's a second example:

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
In this maze, the best paths cost 11048 points; following one such path would look like this:

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#^#
#.#.#.#...#...#^#
#.#.#.#.###.#.#^#
#>>v#.#.#.....#^#
#^#v#.#.#.#####^#
#^#v..#.#.#>>>>^#
#^#v#####.#^###.#
#^#v#..>>>>^#...#
#^#v###^#####.###
#^#v#>>^#.....#.#
#^#v#^#####.###.#
#^#v#^........#.#
#^#v#^#########.#
#S#>>^..........#
#################
Note that the path shown above includes one 90 degree turn as the very first move, rotating the Reindeer from facing East to facing North.

Analyze your map carefully. What is the lowest score a Reindeer could possibly get?


```

In [1]:
from heapq import heappop, heappush

def parse_input(file_name):
    with open(file_name, 'r') as f:
        maze = [list(line.strip()) for line in f]
    start = None
    end = None
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'S':
                start = (i, j)
            elif cell == 'E':
                end = (i, j)
    return maze, start, end

def is_valid(maze, x, y):
    return 0 <= x < len(maze) and 0 <= y < len(maze[0]) and maze[x][y] != '#'

def reindeer_maze(file_name):
    maze, start, end = parse_input(file_name)

    # Directions: (dx, dy) and direction names
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # East, South, West, North
    direction_names = ['E', 'S', 'W', 'N']

    # Priority queue for A* search
    pq = []
    heappush(pq, (0, start[0], start[1], 0))  # (cost, x, y, direction_index)

    # Visited set with state (x, y, direction_index)
    visited = set()

    while pq:
        cost, x, y, direction = heappop(pq)

        if (x, y, direction) in visited:
            continue
        visited.add((x, y, direction))

        # If we reach the end, return the cost
        if (x, y) == end:
            return cost

        # Try moving forward
        nx, ny = x + directions[direction][0], y + directions[direction][1]
        if is_valid(maze, nx, ny):
            heappush(pq, (cost + 1, nx, ny, direction))

        # Try turning left or right
        for turn in [-1, 1]:
            new_direction = (direction + turn) % 4
            heappush(pq, (cost + 1000, x, y, new_direction))

    return -1  # Shouldn't reach here if there's a valid path

# Solve the problem
file_name = "input.txt"
print("Lowest score:", reindeer_maze(file_name))


Lowest score: 73432


```
--- Part Two ---
Now that you know what the best paths look like, you can figure out the best spot to sit.

Every non-wall tile (S, ., or E) is equipped with places to sit along the edges of the tile. While determining which of these tiles would be the best spot to sit depends on a whole bunch of factors (how comfortable the seats are, how far away the bathrooms are, whether there's a pillar blocking your view, etc.), the most important factor is whether the tile is on one of the best paths through the maze. If you sit somewhere else, you'd miss all the action!

So, you'll need to determine which tiles are part of any best path through the maze, including the S and E tiles.

In the first example, there are 45 tiles (marked O) that are part of at least one of the various best paths through the maze:

###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############
In the second example, there are 64 tiles that are part of at least one of the best paths:

#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################
Analyze your map further. How many tiles are part of at least one of the best paths through the maze?
```

In [14]:
from collections import deque
from typing import List, Set, Tuple

class MazeSolver:
    def __init__(self, maze: List[List[str]]):
        self.maze = maze
        self.height = len(maze)
        self.width = len(maze[0])
        self.start, self.end = self._find_start_end()

    def _find_start_end(self) -> Tuple[Tuple[int, int], Tuple[int, int]]:
        start = end = None
        for y in range(self.height):
            for x in range(self.width):
                if self.maze[y][x] == 'S':
                    start = (y, x)
                elif self.maze[y][x] == 'E':
                    end = (y, x)
        return start, end

    def find_optimal_tiles(self) -> int:
        # Directions: right, down, left, up
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

        # First pass: Find minimum score to end from each position and direction
        best_scores = {}  # (y, x, dir) -> min_score
        queue = deque([(self.start[0], self.start[1], 0, 0)])  # y, x, dir, score
        min_end_score = float('inf')

        while queue:
            y, x, dir, score = queue.popleft()

            if score >= min_end_score:
                continue

            state = (y, x, dir)
            if state in best_scores and best_scores[state] <= score:
                continue
            best_scores[state] = score

            # Found end
            if (y, x) == self.end:
                min_end_score = min(min_end_score, score)
                continue

            # Try moving forward
            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and
                self.maze[ny][nx] != '#'):
                queue.append((ny, nx, dir, score + 1))

            # Try turning
            queue.append((y, x, (dir - 1) % 4, score + 1000))
            queue.append((y, x, (dir + 1) % 4, score + 1000))

        # Second pass: Find tiles that are part of optimal paths
        optimal_tiles = set()
        visited = set()
        queue = deque([(self.start[0], self.start[1], 0, 0, {(self.start[0], self.start[1])})])

        while queue:
            y, x, dir, score, path = queue.popleft()

            if score > min_end_score:
                continue

            state = (y, x, dir)
            if score > best_scores.get(state, float('inf')):
                continue

            if (y, x) == self.end and score == min_end_score:
                optimal_tiles.update(path)
                continue

            # Try moving forward
            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and
                self.maze[ny][nx] != '#'):
                new_path = path | {(ny, nx)}
                queue.append((ny, nx, dir, score + 1, new_path))

            # Try turning
            queue.append((y, x, (dir - 1) % 4, score + 1000, path.copy()))
            queue.append((y, x, (dir + 1) % 4, score + 1000, path.copy()))

        return len(optimal_tiles)

def visualize_path(maze: List[List[str]], optimal_tiles: Set[Tuple[int, int]]) -> None:
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if (y, x) in optimal_tiles:
                if maze[y][x] in ['S', 'E']:
                    print(maze[y][x], end='')
                else:
                    print('O', end='')
            else:
                print(maze[y][x], end='')
        print()

def main():
    with open('input.txt', 'r') as f:
        sample_maze = f.read()
    sample_maze = sample_maze.strip()

    maze = [list(line) for line in sample_maze.splitlines()]
    solver = MazeSolver(maze)
    result = solver.find_optimal_tiles()
    print(f"Number of tiles in optimal paths: {result}")

if __name__ == "__main__":
    main()

Number of tiles in optimal paths: 496
